In [2]:
from freespeech.api import transcript

url = "https://docs.google.com/document/d/11NtPE4-TsI8tljKSvW4a6oBpfwg3Ro5NQEmx-Ib6HWI/edit#"

len(" ".join(" ".join(event.chunks) for event in (await transcript.load(url)).events))

2875

In [3]:
2875 / 28

102.67857142857143